In [1]:
import tensorflow as tf
import time
from tensorflow.contrib import rnn
print(tf.__version__)

1.12.0


In [2]:
import mnistdata
mnist = mnistdata.read_data_sets("data/mnist", one_hot=True, reshape=False)

Successfully unzipped train-images-idx3-ubyte.gz
Successfully unzipped train-labels-idx1-ubyte.gz
Successfully unzipped t10k-images-idx3-ubyte.gz
Successfully unzipped t10k-labels-idx1-ubyte.gz


In [3]:
print(mnist.train.images.shape)
print(mnist.test.labels.shape)

(60000, 28, 28, 1)
(10000, 10)


In [4]:
X = tf.placeholder(tf.float32, [None, 28, 28], "X")
Y_ = tf.placeholder(tf.float32, [None, 10], "Y_")

W = tf.Variable(tf.truncated_normal([128, 10]))
b = tf.Variable(tf.zeros([10]))

In [5]:
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(128)
outputs, states = tf.nn.dynamic_rnn(rnn_cell, X, dtype=tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
Y = tf.matmul(outputs, W) + b

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [6]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Y, labels=Y_))
# cross_entropy = tf.reduce_mean(cross_entropy)*100

correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [9]:
start = time.clock()
test_X, test_Y = mnist.test.next_batch(10000)
test_X = test_X.reshape([10000, 28, 28])
for i in range(2000):
    batch_X, batch_Y = mnist.train.next_batch(100)
    batch_X = batch_X.reshape([100, 28, 28])
    if i % 500 == 0:
        train_accuracy = accuracy.eval(session=sess, feed_dict={X:batch_X, Y_:batch_Y})
        test_accuracy = accuracy.eval(session=sess, feed_dict={X:test_X, Y_:test_Y})
        print("step %5d, train_accuracy = %.4g %% test_accuracy = %.4g %%" % (i, train_accuracy*100, test_accuracy*100))
    train_step.run(session=sess, feed_dict={X:batch_X, Y_:batch_Y})
print("training time: ", time.clock()-start, " s")

step     0, train_accuracy = 81 % test_accuracy = 76.32 %
step   500, train_accuracy = 83 % test_accuracy = 86.55 %
step  1000, train_accuracy = 96 % test_accuracy = 90.46 %
step  1500, train_accuracy = 93 % test_accuracy = 91.94 %
